In [ ]:
import numpy as np
def circle(nb_pts,radius):
    Lx = []
    Ly = []
    for i in range(nb_pts):
        Lx.append(radius*np.cos(2*np.pi*i/nb_pts))
        Ly.append(radius*np.sin(2*np.pi*i/nb_pts))
    return Lx, Ly

In [ ]:
rStep = 0.2
nb0 = int(2*np.pi*(1 - rStep)/rStep)
delta_alpha = 2*np.pi/nb0 # used to compute the mapping of the boundary faces and edges
outer_radius = 1
x1_radius = outer_radius - rStep
x,y = circle(nb0,1)
xL = [x]
yL = [y]
for i in range(1,int(1/rStep)):
    x,y = circle(nb0,1-i*rStep)
    xL.append(x)
    yL.append(y)
    nb0 -= 6

xL.append([0.])
yL.append([0.])

In [ ]:
xCols = xL[0].copy()
yCols = yL[0].copy()
for x in xL[1:]:
    xCols += x
    xCols += x
for y in yL[1:]:
    yCols += y
    yCols += y
import matplotlib.pyplot as plt
valL = [1]
for i in range(1,len(xCols)):
    valL.append(valL[-1]+1)
fig, ax = plt.subplots(figsize=(12, 12))
ax.scatter(xCols,yCols)

In [ ]:
import json
M = dict([('Map',{'Outer_dimension':3,'Arguments':[None,[42]],'Arguments_metric':[None,None]}),('Dimension',2),('Cells',[[],[],[]])])
def add_vertex_both(x,y):
    ax.text(x,y,'{}'.format(len(M['Cells'][0])),color='red',fontsize='small')
    M['Cells'][0].append({'Map_ids':[0,1],'Location':[[x,y],[x,y]]})       
    
def add_vertex(x,y,map_id):
    ax.text(x+0.05*map_id,y,'{}'.format(len(M['Cells'][0])),color='red',fontsize='small')
    M['Cells'][0].append({'Map_ids':[map_id],'Location':[[x,y]]})
             
def add_edge(v1,v2,map_id):
    ax.text((xCols[v1]+xCols[v2])/2.+0.05*map_id,(yCols[v1]+yCols[v2])/2.,'{}'.format(len(M['Cells'][1])),fontsize='small')
    M['Cells'][1].append({'Map_ids':[map_id],'Mappings':[0],'Ref_elem':None,'Boundary':[v1,v2]})
    
def add_face(edges,map_id):
    # plot
    xmid = 0.
    ymid = 0.
    for i in edges:
        xmid += xCols[M['Cells'][1][i]['Boundary'][0]]
        xmid += xCols[M['Cells'][1][i]['Boundary'][1]]
        ymid += yCols[M['Cells'][1][i]['Boundary'][0]]
        ymid += yCols[M['Cells'][1][i]['Boundary'][1]]
    xmid = xmid/(2.*len(edges))
    ymid = ymid/(2.*len(edges))
    ax.text(xmid+0.05*map_id,ymid,'{}'.format(len(M['Cells'][2])),color='blue',fontsize='small')
    # add data
    M['Cells'][2].append({'Map_ids':[map_id],'Mappings':[0],'Ref_elem':None,'Boundary':edges.copy()})


In [ ]:
def add_edge_circle(v1,v2,shift_step):
    extra = shift_step*delta_alpha - (np.pi-delta_alpha)/2.
    ax.text((xCols[v1]+xCols[v2])/2.,(yCols[v1]+yCols[v2])/2.,'{}'.format(len(M['Cells'][1])),fontsize='small')
    base_ratio = np.cos((np.pi-delta_alpha)/2.)/np.sin((np.pi-delta_alpha)/2.)
    M['Cells'][1].append({'Map_ids':[0,1],'Mappings':[1,1],
                          'Mappings_extra_args':[[base_ratio,outer_radius,
                                                  np.cos(extra),np.sin(extra)],
                                                [base_ratio,outer_radius,
                                                  np.cos(extra),np.sin(extra)]],
                          'Ref_elem':[[[-0.5],[0.5]]],'Boundary':[v1,v2]})
def add_face_circle(edges,shift_step,map_id):
    # plot
    xmid = 0.
    ymid = 0.
    for i in edges:
        xmid += xCols[M['Cells'][1][i]['Boundary'][0]]
        xmid += xCols[M['Cells'][1][i]['Boundary'][1]]
        ymid += yCols[M['Cells'][1][i]['Boundary'][0]]
        ymid += yCols[M['Cells'][1][i]['Boundary'][1]]
    xmid = xmid/(2.*len(edges))
    ymid = ymid/(2.*len(edges))
    ax.text(xmid+0.05*map_id,ymid,'{}'.format(len(M['Cells'][2])),color='blue',fontsize='small')
    # add data
    extra = shift_step*delta_alpha - (np.pi-delta_alpha)/2.
    ref_alpha_l = (np.pi+delta_alpha)/2.
    ref_alpha_r = (np.pi-delta_alpha)/2.
    v1 = [-0.5,0]
    v2 = [0.5,0]
    v3 = [0.5,1]
    v4 = [-0.5,1]
    M['Cells'][2].append({'Map_ids':[map_id],'Mappings':[1],
                          'Mappings_extra_args':[[outer_radius*np.cos((np.pi-delta_alpha)/2.),
                                                  x1_radius*np.cos((np.pi-delta_alpha)/2.),
                                                  outer_radius*np.sin((np.pi-delta_alpha)/2.),
                                                  x1_radius*np.sin((np.pi-delta_alpha)/2.),
                                                  np.cos(extra),np.sin(extra)]],
                          'Ref_elem':[[v1,v2,v3],[v1,v3,v4]],
                          'Boundary':edges.copy()})      


In [ ]:
# Insert all vertices
for i in range(len(xL[0])):
    add_vertex_both(xL[0][i],yL[0][i])
for j in range(1,len(xL)):
    for i in range(len(xL[j])):
        add_vertex(xL[j][i],yL[j][i],0)
    for i in range(len(xL[j])):
        add_vertex(xL[j][i],yL[j][i],1)

In [ ]:
fig

In [ ]:
offsetVLayer = [[0,0]]
for j in range(1,len(xL)):
    offsetVLayer.append([offsetVLayer[-1][1]+len(xL[j-1]),0])
    offsetVLayer[-1][1] = offsetVLayer[-1][0] + len(xL[j])
offsetELayer = offsetVLayer[:-1].copy()
offsetECross = [[offsetELayer[-1][1]+len(xL[-2]),offsetELayer[-1][1]+len(xL[-2])+len(xL[1])]]
for j in range(2,len(xL)-1):
    offsetECross.append([offsetECross[-1][1]+len(xL[j-1]),0])
    offsetECross[-1][1] = offsetECross[-1][0] + len(xL[j])

In [ ]:
# Insert boundary layer edges
for i in range(len(xL[0])-1):
    add_edge_circle(i+1,i,i) # The mapping is clockwise, and the order must match
add_edge_circle(0,len(xL[0])-1,len(xL[0])-1)
# Flat circle layer
for j in range(1,len(offsetELayer)):
    for side in range(2):
        for i in range(len(xL[j])-1):
            add_edge(i+offsetVLayer[j][side],i+offsetVLayer[j][side]+1,side)
        add_edge(len(xL[j])-1+offsetVLayer[j][side],offsetVLayer[j][side],side)


In [ ]:
#insert the cross edges on the outer layer
for side in range(2):
    for i in range(len(xL[0])):
        add_edge(offsetVLayer[0][side] + i, offsetVLayer[1][side]+i,side)
#insert the rest of the cross edges, mapping to the closest angle
for j in range(1,len(xL)-2):
    nbOut = len(xL[j])
    nbIn = len(xL[j+1])
    for side in range(2):
        for i in range(nbIn):
            add_edge(offsetVLayer[j][side] + round(i*nbOut/nbIn),offsetVLayer[j+1][side]+i,side)
        for i in range(nbIn-1):
            add_face([offsetELayer[j+1][side]+i,offsetECross[j][side]+i] + 
                      list(range(offsetELayer[j][side] + round(i*nbOut/nbIn),
                                 offsetELayer[j][side] + round((i+1)*nbOut/nbIn)))
                      + [offsetECross[j][side]+i+1],side)
        add_face([offsetELayer[j+1][side]+nbIn-1,offsetECross[j][side]+nbIn-1] + 
                 list(range(offsetELayer[j][side] + round((nbIn-1)*nbOut/nbIn),
                            offsetELayer[j][side] + nbOut))
                + [offsetECross[j][side]],side)
#insert the curved faces
for side in range(2):
    for i in range(len(xL[0])-1):
        add_face_circle([offsetELayer[1][side]+i, offsetECross[0][side]+i,
                         offsetELayer[0][side]+i, offsetECross[0][side]+i+1],
                        i,side)
    add_face_circle([offsetELayer[1][side]+len(xL[0])-1, offsetECross[0][side]+len(xL[0])-1,
                         offsetELayer[0][side]+len(xL[0])-1, offsetECross[0][side]],
                        len(xL[0])-1,side)
#fill the center
offsetE  = offsetECross[-1][1]+len(xL[-2])
for side in range(2):
    for i in range(len(xL[-2])):
        add_edge(offsetVLayer[-2][side]+i,offsetVLayer[-1][side],side)
    for i in range(len(xL[-2])-1):
        add_face([offsetE + len(xL[-2])*side + i, offsetELayer[-1][side]+i, offsetE + len(xL[-2])*side + i + 1],side)
    add_face([offsetE + len(xL[-2])*side + len(xL[-2])-1, offsetELayer[-1][side]+len(xL[-2])-1, offsetE + len(xL[-2])*side],side)
    


In [ ]:
fig

In [ ]:
M

In [ ]:
with open('./{}_circle.json'.format(len(xL)),'w') as f:
    json.dump(M,f, indent = 1)